In [19]:
import torch
import torch.optim as optim
torch.manual_seed(1)

#데이터 가져오기
import numpy as np 
import pandas as pd

train_data=pd.read_csv('train.csv')
train_data

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,orange
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,banana
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,mango
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,grapes


In [20]:
#데이터 가공

GrainLabel=train_data.groupby('label')
GrainLabel.groups.keys()

dict_keys(['apple', 'banana', 'blackgram', 'chickpea', 'coconut', 'coffee', 'cotton', 'grapes', 'jute', 'kidneybeans', 'lentil', 'maize', 'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange', 'papaya', 'pigeonpeas', 'pomegranate', 'rice', 'watermelon'])

In [21]:
num=0
for i in GrainLabel.groups.keys():
    train_data.loc[train_data['label']==i,'label']=num
    num+=1

train_data

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,13
1,18,27,41,22.365094,92.308824,7.175344,104.821633,19
2,61,68,50,35.214628,91.497251,6.793245,243.074507,17
3,26,72,22,28.767949,37.577921,4.674942,91.720849,18
4,2,140,197,22.697801,92.822234,5.534567,105.050823,0
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,16
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,1
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,12
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,7


In [22]:
x_train=train_data.copy()
x_train=x_train.drop('label',axis=1)
x_train = torch.FloatTensor(x_train.values)


y_train=train_data['label'].copy()
y_train = torch.FloatTensor(y_train.values)

print(x_train)
print(y_train)

tensor([[ 22.0000,  36.0000,  16.0000,  ...,  50.7715,   8.1842,  64.5856],
        [ 18.0000,  27.0000,  41.0000,  ...,  92.3088,   7.1753, 104.8216],
        [ 61.0000,  68.0000,  50.0000,  ...,  91.4973,   6.7932, 243.0745],
        ...,
        [ 11.0000,  36.0000,  31.0000,  ...,  51.7797,   6.4754, 100.2586],
        [ 11.0000, 124.0000, 204.0000,  ...,  80.0663,   6.3611,  71.4004],
        [ 32.0000,  78.0000,  22.0000,  ...,  62.3556,   7.0070,  53.4091]])
tensor([13., 19., 17.,  ..., 12.,  7., 10.])


In [23]:
y_train=torch.tensor(y_train,dtype=torch.int64)

C:\Users\bjw01\AppData\Local\Temp\ipykernel_29176\3396355151.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train=torch.tensor(y_train,dtype=torch.int64)


In [24]:
import torch.nn.functional as F

nb_class = len(GrainLabel.groups.keys())
nb_data = len(y_train)

W = torch.zeros((7, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.01)
nb_epochs = 10000

for epoch in range(nb_epochs + 1):

    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    cost = F.cross_entropy((x_train.matmul(W) + b), y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 3.091042
Epoch  100/10000 Cost: 1.889747
Epoch  200/10000 Cost: 1.740394
Epoch  300/10000 Cost: 0.447388
Epoch  400/10000 Cost: 0.502795
Epoch  500/10000 Cost: 1.401498
Epoch  600/10000 Cost: 0.433352
Epoch  700/10000 Cost: 0.425114
Epoch  800/10000 Cost: 0.357213
Epoch  900/10000 Cost: 0.396209
Epoch 1000/10000 Cost: 0.527783
Epoch 1100/10000 Cost: 0.453413
Epoch 1200/10000 Cost: 0.440164
Epoch 1300/10000 Cost: 0.452092
Epoch 1400/10000 Cost: 0.161026
Epoch 1500/10000 Cost: 0.199855
Epoch 1600/10000 Cost: 0.433661
Epoch 1700/10000 Cost: 0.210854
Epoch 1800/10000 Cost: 0.185197
Epoch 1900/10000 Cost: 0.199226
Epoch 2000/10000 Cost: 0.199413
Epoch 2100/10000 Cost: 0.420230
Epoch 2200/10000 Cost: 0.127363
Epoch 2300/10000 Cost: 0.177838
Epoch 2400/10000 Cost: 0.184963
Epoch 2500/10000 Cost: 0.186398
Epoch 2600/10000 Cost: 0.190906
Epoch 2700/10000 Cost: 0.191535
Epoch 2800/10000 Cost: 0.191032
Epoch 2900/10000 Cost: 0.193167
Epoch 3000/10000 Cost: 0.516075
Epoch 31

In [25]:
# 학습된 W,b를 통한 클래스 예측
hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
predict = torch.argmax(hypothesis, dim=1)

print(hypothesis)
print(predict)
print(y_train)


# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))



tensor([[1.4753e-10, 2.3385e-16, 1.0682e-03,  ..., 7.9762e-12, 1.3289e-34,
         1.3710e-14],
        [1.3401e-08, 1.3949e-29, 5.7179e-19,  ..., 9.9912e-01, 4.9407e-34,
         1.4342e-19],
        [1.1019e-06, 3.9025e-16, 2.7306e-10,  ..., 7.2350e-16, 3.5602e-05,
         1.3115e-15],
        ...,
        [1.4460e-05, 1.2092e-19, 1.0913e-08,  ..., 8.4050e-09, 6.9649e-29,
         5.0314e-18],
        [7.5087e-04, 0.0000e+00, 0.0000e+00,  ..., 3.2931e-35, 0.0000e+00,
         0.0000e+00],
        [6.8085e-11, 6.1797e-15, 9.4251e-03,  ..., 1.3043e-21, 0.0000e+00,
         5.1161e-23]], grad_fn=<SoftmaxBackward0>)
tensor([13, 19, 17,  ..., 12,  7, 10])
tensor([13, 19, 17,  ..., 12,  7, 10])
tensor([True, True, True,  ..., True, True, True])
The model has an accuracy of 96.55% for the training set.


In [26]:
#테스트 데이터 가져오기
test_data=pd.read_csv('test.csv')
test_data

,N,P,K,temperature,humidity,ph,rainfall
0,101,17,47,29.494014,94.729813,6.185053,26.308209
1,98,8,51,26.179346,86.522581,6.259336,49.430510
2,59,62,49,43.360515,93.351916,6.941497,114.778071
3,44,60,55,34.280461,90.555616,6.825371,98.540477
4,30,137,200,22.914300,90.704756,5.603413,118.604465
...,...,...,...,...,...,...,...
545,100,32,26,25.234661,57.531615,6.043486,124.226174
546,50,46,52,31.182984,90.216469,6.734006,54.018724
547,30,65,82,20.714244,15.278241,7.103798,76.778887
548,93,26,27,24.592457,56.468296,7.288212,137.704405


In [27]:
#정확도 측정
test_data = torch.FloatTensor(test_data.values)
hypothesis = F.softmax(test_data.matmul(W) + b, dim=1)
predict = torch.argmax(hypothesis, dim=1)

print(predict)

tensor([15, 21, 17, 17,  0, 12,  0, 13, 14, 10,  2,  4, 19,  8,  4, 19,  0,  6,
        17, 15,  5, 17, 16, 17,  3,  8, 14, 16, 18, 20, 19, 13, 20, 10, 20,  2,
         8,  3,  3,  9, 17, 12,  2, 11, 14, 11, 18,  4, 15, 11, 13,  5,  7, 14,
         5,  9,  6,  0,  1, 10, 21,  4, 10, 16, 17,  2, 16, 20, 15, 18, 15,  4,
        20,  1,  2, 17,  1,  6, 21, 16,  5,  3, 20, 13, 16, 12,  5, 13,  2, 19,
        11, 13,  6, 17, 18, 13,  9,  5,  2, 10,  4, 20, 16, 15, 21,  9, 21,  1,
        18, 13,  1,  8,  6, 19, 18,  3, 11,  4, 19, 20, 18,  7,  2,  4,  3, 13,
         4, 11,  1, 13,  1,  9, 19,  3,  4, 16, 18,  1,  1,  0,  9, 15, 14, 13,
         4, 11,  0,  4,  9, 17, 14, 10, 21, 14,  2, 18, 18,  9, 11,  8,  3,  0,
        16, 11, 20,  4,  7, 10, 21,  7,  7,  2, 19,  3,  4, 11, 10,  7, 21,  8,
         5,  5,  9,  8, 13,  9,  1,  9,  4, 17, 17, 14, 12, 19, 21,  9, 11,  0,
         2,  3,  7,  7,  1,  6, 20, 19, 14,  1, 20, 14, 11,  3,  3,  3,  0, 20,
         9, 17,  5,  2,  9, 12, 12,  4, 

In [28]:
Label_list=[i for i in GrainLabel.groups.keys()]
print(Label_list)

['apple', 'banana', 'blackgram', 'chickpea', 'coconut', 'coffee', 'cotton', 'grapes', 'jute', 'kidneybeans', 'lentil', 'maize', 'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange', 'papaya', 'pigeonpeas', 'pomegranate', 'rice', 'watermelon']


In [29]:
Id=[]
Label=[]

for i in range(len(predict)):
    Id.append(i)
    Label.append(Label_list[predict[i]])

In [30]:
submit = pd.DataFrame({'id':Id,
                       'label':Label})
submit.to_csv("submit.csv",index=None)